In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/final-data/train_final.csv
/kaggle/input/final-data/test_final.csv


In [2]:
# Reading the test data files
train_final_merged = pd.read_csv('/kaggle/input/final-data/train_final.csv')
test_final_merged = pd.read_csv('/kaggle/input/final-data/test_final.csv')

In [3]:
# checking the final merged test dataset
test_final_merged.head()

,Unnamed: 0,farm_id,ingredient_type,id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour
0,0,fid_110884,ing_w,0,new,7,690.455096,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
1,1,fid_90053,ing_w,1,new,7,252.696160,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
2,2,fid_17537,ing_w,2,moderate,7,499.446528,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
3,3,fid_110392,ing_w,3,moderate,7,2200.407555,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
4,4,fid_62402,ing_w,4,moderate,7,10833.140121,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0


In [4]:
# checking the  data types of final merged tarin dataset
train_final_merged.dtypes

Unnamed: 0                      int64
farm_id                        object
ingredient_type                object
yield                         float64
operations_commencing_year     object
num_processing_plants           int64
farm_area                     float64
farming_company                object
deidentified_location          object
temp_obs                      float64
cloudiness                    float64
wind_direction                float64
pressure_sea_level            float64
precipitation                 float64
wind_speed                    float64
month                           int64
day                             int64
hour                            int64
dtype: object

In [5]:
train_ing_z = train_final_merged[train_final_merged['ingredient_type'] == 'ing_z']

In [6]:
# Split the dataset into three equal parts
train_ing_z_splits = np.array_split(train_ing_z, 3)

# Access and use the first split as a separate dataset
train_z = train_ing_z_splits[0]

In [7]:
test_ing_z = test_final_merged[test_final_merged['ingredient_type'] == 'ing_z']

In [8]:
# Split the dataset into three equal parts
test_ing_z_splits = np.array_split(test_ing_z, 3)

# Access and use the first split as a separate dataset
test_z = test_ing_z_splits[0]

In [9]:
test_z.head()

,Unnamed: 0,farm_id,ingredient_type,id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour
1960832,1960832,fid_87942,ing_z,2260082,new,8,499.260722,Obery Farms,location 959,6.7,2.0,200.0,1024.1,0.0,5.1,1,1,0
1960836,1960836,fid_66870,ing_z,2260086,old,10,5295.006485,Obery Farms,location 959,6.7,2.0,200.0,1024.1,0.0,5.1,1,1,0
1960840,1960840,fid_66062,ing_z,2260090,new,10,2992.034018,Obery Farms,location 959,6.7,2.0,200.0,1024.1,0.0,5.1,1,1,0
1960842,1960842,fid_75323,ing_z,2260092,old,13,9334.986343,Obery Farms,location 959,6.7,2.0,200.0,1024.1,0.0,5.1,1,1,0
1960844,1960844,fid_75397,ing_z,2260094,old,17,12976.969749,Obery Farms,location 959,6.7,2.0,200.0,1024.1,0.0,5.1,1,1,0


In [10]:
# keep a copy of test w we need it in creating submission file
import copy
tes = copy.deepcopy(test_z)

* farm_id is unique just like my id. my performance is measured based on my skills on based on my id right! same here the prediction of yield is depend on loaction and weather condtions on id. so let's drop it


In [11]:
# dropping the farm_id and num_processing pants from test and train datasets
train_z.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)
test_z.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)

In [12]:
# assigning object columns in category columns
cat_cols=['ingredient_type','deidentified_location',
          'operations_commencing_year','farming_company']
cat_cols

['ingredient_type',
 'deidentified_location',
 'operations_commencing_year',
 'farming_company']

In [13]:
# converting object columns to category columns
train_z[cat_cols] = train_z[cat_cols].astype("category")
test_z[cat_cols] = test_z[cat_cols].astype("category")

# Model Building and Predictions

In [14]:
# deviding the data into x and y
from sklearn.model_selection import train_test_split
X = train_z.drop(['yield'] , axis = 1)
y = train_z['yield']

In [15]:
# splitting the data into train and test in the ration of 70% & 30%
X_train, X_test, y_train, y_test =   train_test_split(X, y, test_size=0.30,random_state = 123)

In [16]:
# checking the shapes
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(300474, 15) (128775, 15)
(300474,) (128775,)


<div class="alert alert-block alert-info"><b> 
  Dummifing categorical features into numerical using get dummies
</b></div> 

In [17]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
X_train.deidentified_location = label.fit_transform(X_train.deidentified_location)
X_test.deidentified_location = label.transform(X_test.deidentified_location)
X_train.farming_company = label.fit_transform(X_train.farming_company)
X_test.farming_company = label.transform(X_test.farming_company)

In [18]:
## Convert Categorical Columns to numerical
X_train = pd.get_dummies(X_train, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

X_test = pd.get_dummies(X_test, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [19]:
# scaling the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_cols = ["farm_area", "wind_speed", "precipitation", "temp_obs",
            "pressure_sea_level","wind_direction",'num_processing_plants']

# scale on train
X_train[num_cols] =scaler.fit_transform(X_train[num_cols])

# scale on test
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [20]:
# checking the x train data
X_train.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
3353415,0.0,0.410497,4,0,0.455969,2.0,0.805556,0.484765,0.007519,0.529730,5,7,3,0,0
2204005,0.4,0.156571,2,1,0.471624,2.0,0.750000,0.689751,0.007519,0.221622,7,17,8,1,0
4012012,0.0,0.385429,1,0,0.706458,4.0,0.416667,0.448753,0.007519,0.081081,8,8,14,0,1
3561389,0.0,0.193965,2,0,0.837573,0.0,0.638889,0.452909,0.007519,0.140541,6,5,18,0,0
3167539,0.0,0.317951,2,0,0.336595,2.0,0.638889,0.596953,0.135338,0.529730,4,10,20,1,0


In [21]:
# checking the x test data
X_test.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
2032491,0.2,0.095465,5,1,0.277886,0.0,0.555556,0.591413,0.007519,0.248649,3,25,20,1,0
3128896,0.0,0.455231,4,0,0.489237,2.0,0.333333,0.562327,0.007519,0.113514,4,5,8,0,0
3817005,0.0,0.193361,1,0,0.892368,0.0,0.527778,0.415512,0.007519,0.113514,7,11,23,0,1
2684123,0.0,0.147761,1,0,0.250489,2.0,0.805556,0.583102,0.007519,0.275676,2,2,6,0,1
2356625,0.6,0.173605,2,1,0.307241,2.0,0.666667,0.756233,0.007519,0.167568,10,26,8,0,1


In [22]:
# checking the data types x_train data
X_train.dtypes

num_processing_plants             float64
farm_area                         float64
farming_company                     int64
deidentified_location               int64
temp_obs                          float64
cloudiness                        float64
wind_direction                    float64
pressure_sea_level                float64
precipitation                     float64
wind_speed                        float64
month                               int64
day                                 int64
hour                                int64
operations_commencing_year_new      uint8
operations_commencing_year_old      uint8
dtype: object

# Model Building

<div class="alert alert-block alert-info"><b> 
  Linear Regression
</b>

In [23]:
# linear regression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
train_pred =reg.predict(X_train)
test_pred = reg.predict(X_test)
lr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
lr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('RMSE of Linear Regressor on train : ',lr_train_pred)
print('RMSE of Linear Regressor on test : ',lr_test_pred)

RMSE of Linear Regressor on train :  100.67227150153407
RMSE of Linear Regressor on test :  93.1919195091372


<div class="alert alert-block alert-info"><b> 
  Random Forest Regressor
</b></div> 

In [24]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
train_pred= rfr.predict(X_train)
test_pred = rfr.predict(X_test)
rfr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
rfr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('RMSE of RandomForestRegressor on train', rfr_train_pred)
print('RMSE of RandomForestRegressor on test:', rfr_test_pred)

RMSE of RandomForestRegressor on train 24.504755205275178
RMSE of RandomForestRegressor on test: 49.81903707381008


<div class="alert alert-block alert-info"><b> 
  Gradient Boosting Regressor
</b></div> 

In [25]:
from sklearn.ensemble import GradientBoostingRegressor
gbg = GradientBoostingRegressor(learning_rate=0.05,n_estimators=200,random_state=0)
gbg.fit(X_train,y_train)
train_pred=gbg.predict(X_train)
test_pred = gbg.predict(X_test)
gbr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
gbr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('GBR on train : ', gbr_train_pred)
print('GBR on test : ',gbr_test_pred)

GBR on train :  69.59064531120015
GBR on test :  57.852648217868534


<div class="alert alert-block alert-info"><b> 
  XGB Regressor
</b></div> 

In [26]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=500,learning_rate=0.01)
xgb.fit(X_train,y_train)
train_pred=xgb.predict(X_train)
test_pred = xgb.predict(X_test)
xgb_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
xgb_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('XGB Regressor on train : ', xgb_train_pred)
print('XGB Regressor on test : ',xgb_test_pred)

XGB Regressor on train :  60.04953290485411
XGB Regressor on test :  52.457953124841865


<div class="alert alert-block alert-info"><b> 
  Light Gradient Boosting Regressor
</b></div> 

In [27]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(X_train,y_train)
train_pred=lgb.predict(X_train)
test_pred = lgb.predict(X_test)
lgb_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
lgb_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('LGB Regressor on train:', lgb_train_pred)
print('LGB Regressor on test :',lgb_test_pred)

LGB Regressor on train: 59.85907657863313
LGB Regressor on test : 50.11521758803864


<div class="alert alert-block alert-info"><b> 
  Results of training and testing set
</b></div> 

In [28]:
# Creating a dataframe showing the RMSE for both training and testing data
results=pd.DataFrame({
                    'Models':['Linear Regression','GradientBoost','LGBMRegressor','Random Forest',
                              'XGBoost'],
    
                    'Train_RMSE':[lr_train_pred, gbr_train_pred, lgb_train_pred, rfr_train_pred,
                                  xgb_train_pred],
    
                    'Test_RMSE':[lr_test_pred, gbr_test_pred, lgb_test_pred, rfr_test_pred,
                                 xgb_test_pred],
                     })

In [29]:
results.sort_values('Test_RMSE',ascending=True,ignore_index=True)

,Models,Train_RMSE,Test_RMSE
0,Random Forest,24.504755,49.819037
1,LGBMRegressor,59.859077,50.115218
2,XGBoost,60.049533,52.457953
3,GradientBoost,69.590645,57.852648
4,Linear Regression,100.672272,93.191920


## Conclusion

<div class="alert alert-block alert-info"><b> 
   Light gradient-i regressor gave the best results as compared to all the other algorithms we tried.
</b></div> 

In [30]:
#RandomSearchCV designed to find the best parameters to improve our model
#from sklearn.model_selection import RandomizedSearchCV
#rf_grid = RandomForestRegressor()
#rf_param_grid = {"n_estimators" : [100,500,1000],
#                 "max_features" : ['sqrt','log2'],
#                 "max_depth" : [10,20,30],
#                "min_samples_leaf" : [2,3,5,7,10]}
#rf2=RandomizedSearchCV(rf_grid,rf_param_grid,cv=5,n_jobs=-1,n_iter=10)
#rf2.fit(X_train,y_train.values.ravel())
#best_hyper = rf2.best_params_

In [31]:
#best model
#best_model = RandomForestRegressor()
#best_model.fit(X_train,y_train)
#train_pred=rf2.predict(X_train)
#test_pred = rf2.predict(X_test)
#print('RMSE RandomizedSearchCV on train : ', np.sqrt(mean_squared_error(y_train, train_pred)))
#print('RMSE RandomizedSearchCV on test : ', np.sqrt(mean_squared_error(y_test, test_pred)))

## Lets work on test data

In [32]:
test_z.deidentified_location = label.fit_transform(test_z.deidentified_location)
test_z.farming_company = label.fit_transform(test_z.farming_company)

In [33]:
# converting categorical to numerical
test_z = pd.get_dummies(test_z, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [34]:
# scale on test
test_z[num_cols] = scaler.fit_transform(test_z[num_cols])

In [35]:
# selecting remaing columns except id column
fin = test_z.loc[:, test_z.columns != 'id']

In [36]:
# checking the final test dataset dimensions
fin.shape

(430516, 15)

In [37]:
fin.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
1960832,0.1,0.000000,2,1,0.206107,2.0,0.555556,0.77071,0.004255,0.3,1,1,0,1,0
1960836,0.3,0.094227,2,1,0.206107,2.0,0.555556,0.77071,0.004255,0.3,1,1,0,0,1
1960840,0.3,0.048978,2,1,0.206107,2.0,0.555556,0.77071,0.004255,0.3,1,1,0,1,0
1960842,0.6,0.173605,2,1,0.206107,2.0,0.555556,0.77071,0.004255,0.3,1,1,0,0,1
1960844,1.0,0.245163,2,1,0.206107,2.0,0.555556,0.77071,0.004255,0.3,1,1,0,0,1


# Predictions

In [38]:
# predicting the test data with best model
test_pred = lgb.predict(fin)

In [39]:
# creating a dataframe with id column and yield column
sub  = pd.DataFrame({'id': tes.id, 'yield': test_pred})

In [40]:
# checking the shape of the final dataframe 
sub.shape

(430516, 2)

In [41]:
# checking the head of the dataframe
sub.head()

,id,yield
1960832,2260082,12.389789
1960836,2260086,41.578100
1960840,2260090,84.431621
1960842,2260092,57.275001
1960844,2260094,301.900992


In [42]:
# checking the last 5 rows of the dataframe
sub.tail()

,id,yield
4226831,5028851,74.674518
4226834,5028854,162.535683
4226839,5028859,2.201927
4226848,5028868,3.849064
4226851,5028871,1.427313


In [43]:
# creating submission file
sub.to_csv('submission_z1.csv', index = None)